# AAPL Stock Time Series Forecasting

This notebook demonstrates time series forecasting for Apple Inc. (AAPL) stock using various techniques including:
- Data fetching with yfinance
- Exploratory data analysis
- Time series decomposition
- Multiple forecasting models (ARIMA, Prophet, Linear Regression)
- Model evaluation and comparison

## 1. Import Required Libraries

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Time series analysis and forecasting
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("All libraries imported successfully!")

## 2. Fetch AAPL Stock Data

In [ ]:
# Fetch AAPL stock data for the last 3 years to ensure enough data for decomposition
ticker = "AAPL"
end_date = datetime.now()
start_date = end_date - timedelta(days=1095)  # 3 years of data (ensures >504 observations)

# Download the data
aapl = yf.download(ticker, start=start_date, end=end_date)

# Display basic information about the dataset
print(f"Dataset shape: {aapl.shape}")
print(f"Date range: {aapl.index.min()} to {aapl.index.max()}")
print("\nFirst few rows:")
print(aapl.head())

# Display basic statistics
print("\nBasic statistics:")
print(aapl.describe())

## 3. Data Visualization and Exploration

In [ ]:
# Create interactive plot of AAPL stock price
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=aapl.index,
    y=aapl['Close'],
    mode='lines',
    name='Close Price',
    line=dict(color='blue', width=2)
))

fig.update_layout(
    title='AAPL Stock Price Over Time',
    xaxis_title='Date',
    yaxis_title='Price ($)',
    template='plotly_white',
    height=500
)

fig.show()

# Plot volume
fig_volume = go.Figure()
fig_volume.add_trace(go.Bar(
    x=aapl.index,
    y=aapl['Volume'],
    name='Volume',
    marker_color='orange'
))

fig_volume.update_layout(
    title='AAPL Trading Volume Over Time',
    xaxis_title='Date',
    yaxis_title='Volume',
    template='plotly_white',
    height=400
)

fig_volume.show()

## 4. Time Series Decomposition

In [ ]:
# Perform time series decomposition
data_length = len(aapl['Close'])
print(f"Dataset length: {data_length} observations")

# Choose appropriate period based on data length
if data_length >= 504:  # Enough for yearly seasonality (252 trading days * 2)
    period = 252
    seasonality_type = "Yearly (252 trading days)"
elif data_length >= 126:  # Enough for quarterly seasonality
    period = 63  # ~3 months
    seasonality_type = "Quarterly (~63 trading days)"
else:  # Use monthly seasonality
    period = min(21, data_length // 3)  # Monthly or smaller
    seasonality_type = f"Monthly (~{period} trading days)"

print(f"Using {seasonality_type} for decomposition")

decomposition = seasonal_decompose(aapl['Close'], model='multiplicative', period=period)

# Create subplots for decomposition
fig, axes = plt.subplots(4, 1, figsize=(15, 12))

decomposition.observed.plot(ax=axes[0], title='Original Time Series', color='blue')
decomposition.trend.plot(ax=axes[1], title='Trend', color='green')
decomposition.seasonal.plot(ax=axes[2], title=f'Seasonal ({seasonality_type})', color='orange')
decomposition.resid.plot(ax=axes[3], title='Residual', color='red')

plt.tight_layout()
plt.show()

# Check for stationarity using Augmented Dickey-Fuller test
def check_stationarity(timeseries):
    result = adfuller(timeseries.dropna())
    print('Augmented Dickey-Fuller Test:')
    print(f'ADF Statistic: {result[0]:.6f}')
    print(f'p-value: {result[1]:.6f}')
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'\t{key}: {value:.3f}')
    
    if result[1] <= 0.05:
        print("Result: Time series is stationary")
    else:
        print("Result: Time series is non-stationary")

print("Stationarity test for original series:")
check_stationarity(aapl['Close'])

## 5. Data Preprocessing for Forecasting

In [ ]:
# Prepare data for modeling
# Split data into train and test sets (80% train, 20% test)
split_date = aapl.index[int(len(aapl) * 0.8)]
train_data = aapl[aapl.index <= split_date]
test_data = aapl[aapl.index > split_date]

print(f"Training data: {len(train_data)} records")
print(f"Testing data: {len(test_data)} records")
print(f"Split date: {split_date}")

# Focus on Close price for forecasting
train_close = train_data['Close']
test_close = test_data['Close']

# Check if data needs differencing for stationarity
train_diff = train_close.diff().dropna()
print("\nStationarity test for differenced series:")
check_stationarity(train_diff)

## 6. ARIMA Model Forecasting

In [ ]:
# Fit ARIMA model
# Using simple ARIMA(1,1,1) for demonstration
arima_model = ARIMA(train_close, order=(1, 1, 1))
arima_fitted = arima_model.fit()

print("ARIMA Model Summary:")
print(arima_fitted.summary())

# Make predictions
arima_forecast = arima_fitted.forecast(steps=len(test_close))
arima_conf_int = arima_fitted.get_forecast(steps=len(test_close)).conf_int()

# Create forecast plot
plt.figure(figsize=(15, 8))
plt.plot(train_close.index, train_close, label='Training Data', color='blue')
plt.plot(test_close.index, test_close, label='Actual Test Data', color='green')
plt.plot(test_close.index, arima_forecast, label='ARIMA Forecast', color='red', linestyle='--')
plt.fill_between(test_close.index, 
                 arima_conf_int.iloc[:, 0], 
                 arima_conf_int.iloc[:, 1], 
                 color='pink', alpha=0.3, label='Confidence Interval')
plt.title('AAPL Stock Price Forecast - ARIMA Model')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Calculate ARIMA metrics
arima_mse = mean_squared_error(test_close, arima_forecast)
arima_mae = mean_absolute_error(test_close, arima_forecast)
arima_rmse = np.sqrt(arima_mse)

print(f"\nARIMA Model Performance:")
print(f"MSE: {arima_mse:.2f}")
print(f"MAE: {arima_mae:.2f}")
print(f"RMSE: {arima_rmse:.2f}")

## 7. Prophet Model Forecasting

In [ ]:
# Install Prophet if not already installed (uncomment if needed)
# !pip install prophet

from prophet import Prophet

# Prepare data for Prophet (needs 'ds' and 'y' columns)
prophet_data = train_close.reset_index()
prophet_data.columns = ['ds', 'y']

# Initialize and fit Prophet model
prophet_model = Prophet(
    daily_seasonality=True,
    weekly_seasonality=True,
    yearly_seasonality=True,
    changepoint_prior_scale=0.05
)
prophet_model.fit(prophet_data)

# Create future dates for prediction
future_dates = prophet_model.make_future_dataframe(periods=len(test_close))
prophet_forecast = prophet_model.predict(future_dates)

# Extract predictions for test period
prophet_test_forecast = prophet_forecast.tail(len(test_close))

# Plot Prophet forecast
fig = prophet_model.plot(prophet_forecast, figsize=(15, 8))
plt.title('AAPL Stock Price Forecast - Prophet Model')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Plot forecast components
fig_components = prophet_model.plot_components(prophet_forecast, figsize=(15, 10))
plt.tight_layout()
plt.show()

# Calculate Prophet metrics
prophet_predictions = prophet_test_forecast['yhat'].values
prophet_mse = mean_squared_error(test_close, prophet_predictions)
prophet_mae = mean_absolute_error(test_close, prophet_predictions)
prophet_rmse = np.sqrt(prophet_mse)

print(f"\nProphet Model Performance:")
print(f"MSE: {prophet_mse:.2f}")
print(f"MAE: {prophet_mae:.2f}")
print(f"RMSE: {prophet_rmse:.2f}")

## 8. Linear Regression Model

In [ ]:
# Create features for linear regression (using time as feature)
def create_features(data):
    data_reset = data.reset_index()
    data_reset['timestamp'] = pd.to_datetime(data_reset['Date']).astype(int) / 10**9
    return data_reset['timestamp'].values.reshape(-1, 1)

# Prepare features
X_train = create_features(train_close)
y_train = train_close.values
X_test = create_features(test_close)

# Fit linear regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Make predictions
lr_forecast = lr_model.predict(X_test)

# Create forecast plot
plt.figure(figsize=(15, 8))
plt.plot(train_close.index, train_close, label='Training Data', color='blue')
plt.plot(test_close.index, test_close, label='Actual Test Data', color='green')
plt.plot(test_close.index, lr_forecast, label='Linear Regression Forecast', color='purple', linestyle='--')
plt.title('AAPL Stock Price Forecast - Linear Regression Model')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Calculate Linear Regression metrics
lr_mse = mean_squared_error(test_close, lr_forecast)
lr_mae = mean_absolute_error(test_close, lr_forecast)
lr_rmse = np.sqrt(lr_mse)
lr_r2 = r2_score(test_close, lr_forecast)

print(f"\nLinear Regression Model Performance:")
print(f"MSE: {lr_mse:.2f}")
print(f"MAE: {lr_mae:.2f}")
print(f"RMSE: {lr_rmse:.2f}")
print(f"R² Score: {lr_r2:.4f}")

## 9. Model Comparison and Results Summary ##

In [ ]:
# Efficient model comparison and results summary

def calculate_metrics(actual, predicted):
    mse = mean_squared_error(actual, predicted)
    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    return mse, mae, rmse, mape

# Calculate metrics for all models
arima_metrics = calculate_metrics(test_close, arima_forecast)
prophet_metrics = calculate_metrics(test_close.values, prophet_predictions)
lr_metrics = calculate_metrics(test_close, lr_forecast)
lr_r2 = r2_score(test_close, lr_forecast)

# Create summary DataFrame
results_df = pd.DataFrame({
    'Model': ['ARIMA (1,1,1)', 'Prophet', 'Linear Regression'],
    'MSE': [arima_metrics[0], prophet_metrics[0], lr_metrics[0]],
    'MAE': [arima_metrics[1], prophet_metrics[1], lr_metrics[1]],
    'RMSE': [arima_metrics[2], prophet_metrics[2], lr_metrics[2]],
    'MAPE (%)': [arima_metrics[3], prophet_metrics[3], lr_metrics[3]],
    'R² Score': ['-', '-', f"{lr_r2:.4f}"]
})

mean_actual = test_close.mean()
results_df['MAE %'] = (results_df['MAE'] / mean_actual * 100).round(2)
results_df['RMSE %'] = (results_df['RMSE'] / mean_actual * 100).round(2)

print("="*60)
print("AAPL STOCK FORECASTING RESULTS SUMMARY")
print("="*60)
print("\nModel Performance Comparison:")
print(results_df.to_string(index=False))

# Determine best model
best_rmse_idx = results_df['RMSE'].idxmin()
best_mae_idx = results_df['MAE'].idxmin()

print(f"\n📊 PERFORMANCE ANALYSIS:")
best_rmse_val = results_df.iloc[best_rmse_idx]['RMSE']
if hasattr(best_rmse_val, "item"):
    best_rmse_val = best_rmse_val.item()
best_mae_val = results_df.iloc[best_mae_idx]['MAE']
if hasattr(best_mae_val, "item"):
    best_mae_val = best_mae_val.item()
print(f"Best RMSE: {results_df.iloc[best_rmse_idx]['Model']} (RMSE: {best_rmse_val:.2f})")
print(f"Best MAE:  {results_df.iloc[best_mae_idx]['Model']} (MAE: {best_mae_val:.2f})")

# Residuals and APEs
residuals = [test_close - arima_forecast, test_close.values - prophet_predictions, test_close - lr_forecast]
# Convert test_close to numpy array for consistent operations
test_close_values = test_close.values
apes = [np.abs(residual / test_close_values) * 100 for residual in residuals]

# Plotting
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: All forecasts comparison
axes[0,0].plot(train_close.index[-50:], train_close.tail(50), label='Training Data', color='blue', linewidth=2)
axes[0,0].plot(test_close.index, test_close, label='Actual', color='green', linewidth=2)
axes[0,0].plot(test_close.index, arima_forecast, label='ARIMA', color='red', linestyle='--', alpha=0.8)
axes[0,0].plot(test_close.index, prophet_predictions, label='Prophet', color='orange', linestyle='--', alpha=0.8)
axes[0,0].plot(test_close.index, lr_forecast, label='Linear Reg', color='purple', linestyle='--', alpha=0.8)
axes[0,0].set_title('All Models Forecast Comparison')
axes[0,0].set_xlabel('Date')
axes[0,0].set_ylabel('Price ($)')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Plot 2: Model performance metrics
axes[0,1].bar(results_df['Model'], results_df['RMSE'], color=['red', 'orange', 'purple'], alpha=0.7)
axes[0,1].set_title('RMSE Comparison')
axes[0,1].set_ylabel('RMSE')
axes[0,1].tick_params(axis='x', rotation=45)
for i, v in enumerate(results_df['RMSE']):
    axes[0,1].text(i, v + 0.5, f'{v:.1f}', ha='center', va='bottom')

# Plot 3: Residuals analysis
for i, label in enumerate(['ARIMA', 'Prophet', 'Linear Reg']):
    axes[1,0].plot(test_close.index, residuals[i], label=label, alpha=0.7)
axes[1,0].axhline(y=0, color='black', linestyle='-', alpha=0.5)
axes[1,0].set_title('Forecast Residuals')
axes[1,0].set_xlabel('Date')
axes[1,0].set_ylabel('Residual ($)')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Plot 4: Absolute percentage errors
for i, label in enumerate(['ARIMA', 'Prophet', 'Linear Reg']):
    axes[1,1].plot(test_close.index, apes[i], label=label, alpha=0.7)
axes[1,1].set_title('Absolute Percentage Error')
axes[1,1].set_xlabel('Date')
axes[1,1].set_ylabel('APE (%)')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistical analysis of residuals
print(f"\n📈 RESIDUAL ANALYSIS:")
for i, label in enumerate(['ARIMA', 'Prophet', 'Linear Reg']):
    # Convert to numpy array to ensure consistent operations
    residual_values = np.array(residuals[i])
    print(f"{label} Residuals - Mean: {np.mean(residual_values):.2f}, Std: {np.std(residual_values):.2f}")

print(f"\n📊 ADDITIONAL METRICS:")
for i, label in enumerate(['ARIMA', 'Prophet', 'Linear Reg']):
    # Convert to numpy array for consistent operations
    ape_values = np.array(apes[i])
    print(f"{label} MAPE: {np.mean(ape_values):.2f}%")

# Model insights
print(f"\n🔍 KEY INSIGHTS:")
print(f"• Dataset: {len(aapl)} observations over {(aapl.index.max() - aapl.index.min()).days} days")
print(f"• Training period: {len(train_close)} observations ({len(train_close)/len(aapl)*100:.1f}% of data)")
print(f"• Test period: {len(test_close)} observations ({len(test_close)/len(aapl)*100:.1f}% of data)")
mean_actual_scalar = mean_actual.iloc[0] if hasattr(mean_actual, "iloc") else float(mean_actual)
# Ensure scalar values for formatting
min_close = aapl['Close'].min()
max_close = aapl['Close'].max()
if hasattr(min_close, "item"):
    min_close = min_close.item()
if hasattr(max_close, "item"):
    max_close = max_close.item()
